In [1]:
import cvxpy as cp
import numpy as np

In [19]:
T = 3 # Num time periods
S = 5 # Num stations
V = 2 # Num vehicles

C_hat_v = np.ones(V) # Capacity of each vehicle v
d_hat_v_1 = np.array([10, 12]) # Initial num bikes in each vehicle v
d_s_1 = np.array([5, 6, 3, 10, 11])

In [3]:
d_hat = cp.Variable((V, T), name="d_hat") # Num bikes in vehicle v at time t
d = cp.Variable((S, T), name="d")
x_plus = cp.Variable((S, T), name="x_plus") # Num of successful bike trips starting at stations s at time t
x_minus = cp.Variable((S, T), name="x_minus") # Num of successful returns starting at stations s at time t

In [9]:
z = {}
r_plus = {}
r_minus = {}
for t in range(T):
    r_plus[t] = cp.Variable((S, V), name="r_plus-{}".format(t))
    r_minus[t] = cp.Variable((S, V), name="r_minus-{}".format(t))
    z[t] = cp.Variable((S, V), integer=True, name="z-{}".format(t))

In [5]:
d_hat_constraint = [
    d_hat[:, 0] == d_hat_v_1
]
# Other d_hat
d_hat_constraint.extend([
    d_hat[:, t+1] == (
        d_hat[:, t]
        + sum([r_plus[t][s, :] - r_minus[t][s, :] for s in range(S)])
    )

    for t in range(T-1)
])

In [6]:
for i in d_hat_constraint:
    print(str(i))

d_hat[0:2, 0] == [10. 12.]
d_hat[0:2, 1] == d_hat[0:2, 0] + r_plus-0[0, 0:2] + -r_minus-0[0, 0:2] + r_plus-0[1, 0:2] + -r_minus-0[1, 0:2] + r_plus-0[2, 0:2] + -r_minus-0[2, 0:2] + r_plus-0[3, 0:2] + -r_minus-0[3, 0:2] + r_plus-0[4, 0:2] + -r_minus-0[4, 0:2]
d_hat[0:2, 2] == d_hat[0:2, 1] + r_plus-1[0, 0:2] + -r_minus-1[0, 0:2] + r_plus-1[1, 0:2] + -r_minus-1[1, 0:2] + r_plus-1[2, 0:2] + -r_minus-1[2, 0:2] + r_plus-1[3, 0:2] + -r_minus-1[3, 0:2] + r_plus-1[4, 0:2] + -r_minus-1[4, 0:2]


In [7]:
d_constraint = [
    d[:, 0] == d_s_1
]
# Other d
d_constraint.extend([
    d[:, t+1] == (
        d[:, t]
        + sum([r_plus[t][:, v] - r_minus[t][:, v] for v in range(V)])
        - x_plus[:, t] + x_minus[:, t]
    )

    for t in range(T-1)
])

In [8]:
for i in d_constraint:
    print(str(i))

d[0:5, 0] == [ 5.  6.  3. 10. 11.]
d[0:5, 1] == d[0:5, 0] + r_plus-0[0:5, 0] + -r_minus-0[0:5, 0] + r_plus-0[0:5, 1] + -r_minus-0[0:5, 1] + -x_plus[0:5, 0] + x_minus[0:5, 0]
d[0:5, 2] == d[0:5, 1] + r_plus-1[0:5, 0] + -r_minus-1[0:5, 0] + r_plus-1[0:5, 1] + -r_minus-1[0:5, 1] + -x_plus[0:5, 1] + x_minus[0:5, 1]


In [15]:
z_constraint = [
    sum([z[t][s, :] for s in range(S)]) == 1
    for t in range(T)
]

In [16]:
str(z_constraint[0])

'z-0[0, 0:2] + z-0[1, 0:2] + z-0[2, 0:2] + z-0[3, 0:2] + z-0[4, 0:2] == 1.0'

In [24]:
capacity_constraint = [
    r_plus[t][s, :] + r_minus[t][s, :] <= cp.multiply(C_hat_v,  z[t][s, :])
    for s in range(S) for t in range(T)
]

In [27]:
str(capacity_constraint[0])

'r_plus-0[0, 0:2] + r_minus-0[0, 0:2] <= [1. 1.] @ z-0[0, 0:2]'

In [28]:
d_hat_limits = [
    d_hat >= 0
]
d_hat_limits.extend([
    d_hat[:, t] <= C_hat_v
    for t in range(T)
])

In [41]:
f_plus = np.ones((S, T)) # Expected rental demand at station s at time t
f_minus = np.ones((S, T))
x_plus_limits = [
    x_plus >= 0
]
x_plus_limits.extend([
    x_plus <= f_plus
])

In [34]:
str(x_plus_limits[1])

'x_plus <= [[1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]]'

In [35]:
r_plus_limits = [
    0 <= r_plus[t] for t in range(T)
]

In [37]:
str(r_plus_limits[0])

'0.0 <= r_plus-0'

In [38]:
r_minus_limits = [
    r_minus[t][s, :] <= C_hat_v
    for s in range(S) for t in range(T)
]

In [39]:
str(r_minus_limits[0])

'r_minus-0[0, 0:2] <= [1. 1.]'

In [42]:
objective = (
    sum([f_plus - x_plus])
    + sum([f_minus - x_minus])
)

In [47]:
str(sum(objective))

'[[1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]] + -x_plus + [[1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]] + -x_minus[0, 0:3] + [[1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]] + -x_plus + [[1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]] + -x_minus[1, 0:3] + [[1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]] + -x_plus + [[1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]] + -x_minus[2, 0:3] + [[1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]] + -x_plus + [[1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]] + -x_minus[3, 0:3] + [[1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]] + -x_plus + [[1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]\n [1. 1. 1.]] + -x_minus[4, 0:3]'

In [50]:
sum(sum(objective))

Expression(AFFINE, UNKNOWN, ())